In [1]:
import pandas as pd
import sys
sys.path.insert(0, "/media/diskD/EPFL/Fall 2016/ADA/Project/GMR_ADA_Project/EmotionAnalysis")
from DataSchemaExtractionParsing import *
from DataPreProcessing import *
#from SentSemanticModule import *
from SentTweetModule import *
from SentSyntacticModule import *

### Loading Annotated Dataset

In [18]:
plutchik_tweets = pd.read_csv('Data/primary-plutchik-wheel-DFE.csv',encoding="utf-8")
plutchik_tweets = plutchik_tweets[['text','emotion']]
plutchik_tweets.head()

,text,emotion
0,How much of the forecast was genuine and how m...,Neutral
1,I did touch them one time you see but of cours...,Neutral
2,We find that choice theorists admit that they ...,Neutral
3,"Well, here I am with an olive branch.",Neutral
4,"Its rudder and fin were both knocked out, and ...",Neutral


In [19]:
len(plutchik_tweets)

2524

In [20]:
set(plutchik_tweets['emotion'])

{u'Aggression',
 u'Ambiguous',
 u'Anger',
 u'Anticipation',
 u'Awe',
 u'Contempt',
 u'Disapproval',
 u'Disgust',
 u'Fear',
 u'Joy',
 u'Love',
 u'Neutral',
 u'Optimism',
 u'Remorse',
 u'Sadness',
 u'Submission',
 u'Surprise',
 u'Trust'}

In [21]:
selective_8 = ['Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust','Neutra']
plutchik_tweets_8emotions = plutchik_tweets[plutchik_tweets['emotion'].isin(selective_8)]
len(plutchik_tweets_8emotions)

949

### Applying the same pre-processing pipeline to get affective representation:

In [27]:
tweets_no_contractions = replace_contractions(plutchik_tweets_8emotions)

### Tokenization of Sentences into words

In [28]:
tokenized_list = bag_of_word_representation(tweets_no_contractions)

### Part of Speech Tagging:

In [29]:
tagged_tweets = pos_tagging(tokenized_list)

### Dependency Parser:

In [16]:
import spacy
from spacy.symbols import nsubj, advmod, acomp, amod, neg, NOUN, VERB, ADJ, ADV
nlp = spacy.load('en')

In [72]:
docs = []
# Joining text:
tweets_text = []
for i in range(0, len(tokenized_list)):
    space = " "
    tweets_text.append(space.join(tokenized_list[i]))
tweets_text[0].encode("utf-8")
for i in range(0, len(tweets_text)):
    doc = nlp(tweets_text[i])
    docs.append(doc)

In [73]:
new_samples = []
for sample in docs:
    new_samples_sub = []
    for word in sample:
        new_samples_sub.append((unicode(word),word.pos_))
    new_samples.append(new_samples_sub)

### Application of Syntactic Rules

In [74]:
triple_dependencies = []
samples = docs
for i in range(0,len(samples)):
    negated_verbs = []
    negated_nouns = []
    triple_dependencies_sub = []
    for j in range(0,len(samples[i])):
        #print "i="+str(i)
        #print "j="+str(j)
        word = samples[i][j]
        #print word
        triple_dependencies_sub.append((word,word.dep_,word.head))
        # Looking for negation dependency
        if word.dep == neg:
            new_samples[i].append(("not_"+str(word.head),word.head.pos_))
            new_samples[i] = [x for x in new_samples[i] if x != (unicode(word.head),word.head.pos_) and x!=(unicode(word),word.pos_)]
            if word.head.pos == VERB:
                negated_verbs.append(word.head.i)
        # Looking for adjectival complement
        if word.dep == acomp:
            
            if word.head.i in negated_verbs:
                new_samples[i].append(("not_"+str(word),word.pos_))
                new_samples[i] = [x for x in new_samples[i] if x != ("not_"+str(word.head),word.head.pos_) and x != (unicode(word),word.pos_)]
            else: 
                new_samples[i] = [x for x in new_samples[i] if x != (unicode(word.head),word.head.pos_)]
        # Looking for negation in nouns and adjectives
        if word.lemma_ == "no" or word.lemma_ == 'not' or word.lemma_ == "never":
            if word.head.pos == NOUN:
                new_samples[i].append(("not_"+str(word.head),word.head.pos_))
                new_samples[i] = [x for x in new_samples[i] if x != (unicode(word.head),word.head.pos_) and x!=(unicode(word),word.pos_)]
                negated_nouns.append(word.head.i)
        # Looking for adjectival modifier 
        if word.dep == amod:
            if word.head.i in negated_nouns:
                new_samples[i].append(("not_"+str(word),word.pos_))
                new_samples[i] = [x for x in new_samples[i] if x != ("not_"+str(word.head),word.head.pos_) and x != (unicode(word),word.pos_)]
            else:
                new_samples[i] = [x for x in new_samples[i] if x != (unicode(word.head),word.head.pos_)]
        # Looking for adverbial modifier
        if word.dep == advmod:
            new_samples[i] = [x for x in new_samples[i] if x != (unicode(word),word.pos_)]
    triple_dependencies.append(triple_dependencies_sub)

<<<<<<<inside advmod>>>
[(u'Brian', u'PROPN'), (u'as', u'ADP'), (u'decided', u'VERB'), (u'not', u'ADV'), (u'to', u'PART'), (u'sit', u'VERB'), (u'on', u'ADP'), (u'the', u'DET'), (u'fence', u'NOUN')]
<<<<<<<<<inside negation>>>>>>>>>
[(u'Brian', u'PROPN'), (u'as', u'ADP'), (u'decided', u'VERB'), (u'to', u'PART'), (u'on', u'ADP'), (u'the', u'DET'), (u'fence', u'NOUN'), ('not_sit', u'VERB')]
(u'sit', u'VERB')
(u'not', u'ADV')
<<<<<<<inside verb
[(u'Brian', u'PROPN'), (u'as', u'ADP'), (u'decided', u'VERB'), (u'to', u'PART'), (u'on', u'ADP'), (u'the', u'DET'), (u'fence', u'NOUN'), ('not_sit', u'VERB')]
<<<<<<<inside negation noun>>>
[(u'Brian', u'PROPN'), (u'as', u'ADP'), (u'decided', u'VERB'), (u'to', u'PART'), (u'on', u'ADP'), (u'the', u'DET'), (u'fence', u'NOUN'), ('not_sit', u'VERB')]


In [75]:
i = 1
print "\n<<<< Original tweet text >>>\n"
print tweets_text[i]
print "\n<<<< Syntactic dependencies >>>\n"
print triple_dependencies[i]
print "\n<<<< Tweet after applying syntactic Rules >>>\n"
#print new_samples[i]
new_tweet = []
for (word,pos) in new_samples[i]:
    new_tweet.append(word)
print new_tweet


<<<< Original tweet text >>>

Brian as ever decided not to sit on the fence

<<<< Syntactic dependencies >>>

[(Brian, u'nsubj', decided), (as, u'mark', decided), (ever, u'advmod', decided), (decided, u'ROOT', decided), (not, u'neg', sit), (to, u'aux', sit), (sit, u'xcomp', decided), (on, u'prep', sit), (the, u'det', fence), (fence, u'pobj', on)]

<<<< Tweet after applying syntactic Rules >>>

[u'Brian', u'as', u'decided', u'to', u'on', u'the', u'fence', 'not_sit']


### Named Entity Recognition:

In [76]:
import nltk

def extract_entity_names(t):
    non_entity_names = []
    entity_names = []
   
    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))
    else:
        non_entity_names.append(t)
    return non_entity_names

tweet_without_ne = []
for tweet in new_samples:
    nre_tweet = nltk.ne_chunk(tweet, binary = True)
    non_entity_names = []
    for tree in nre_tweet:    
        non_entity_names.extend(extract_entity_names(tree))
    tweet_without_ne.append(non_entity_names)

### Normalizing POS Tags

In [79]:
def normalize_pos_tags_words(tagged_tweets):
    """
    :param tagged_tweets:
    :return: tweets_nava
    """
    tweets_nava = []
    tweets_nava_sub = []
    for i in range(0, len(tagged_tweets)):
        for (word, tag) in tagged_tweets[i]:
            if tag == "NOUN":
                tweets_nava_sub.append((word, 'n'))
            elif tag == 'VERB':
                tweets_nava_sub.append((word, 'v'))
            else: 
                tweets_nava_sub.append((word, tag))
        tweets_nava.append(list(tweets_nava_sub))
        tweets_nava_sub = []
    return tweets_nava
normalized_tags = normalize_pos_tags_words(tweet_without_ne)
normalized_tags[1]

[(u'as', u'ADP'),
 (u'decided', 'v'),
 (u'to', u'PART'),
 (u'on', u'ADP'),
 (u'the', u'DET'),
 (u'fence', 'n'),
 ('not_sit', 'v')]

### Removal of Punctuation and Stop words and Converting to Lower Case and Removal of Other special categories: url, number, username:

In [80]:
from nltk.corpus import stopwords
def eliminate_stop_words_punct(tagged_tweets):
    """
    Elimination of Stop words
    Elimination of Punctuation

    :rtype: object
    :param tagged_tweets:
    :return: tagged_tweets_without
    """
    stop_words = list(set(stopwords.words('english')))
    tagged_tweets_without = []
    for i in range(0, len(tagged_tweets)):
        tagged_tweets_without_sub = []
        for (word, tag) in tagged_tweets[i]:
            if word not in stop_words and word not in ['url','number','username'] and len(word) >= 2:
                tagged_tweets_without_sub.append((word.lower(), tag))
        tagged_tweets_without.append(tagged_tweets_without_sub)
    return tagged_tweets_without
tagged_tweets_without = eliminate_stop_words_punct(normalized_tags)

### Lemmatization:

In [81]:
def lemmatizer(tweets):
    tweets_whole = []
    lmtzr = WordNetLemmatizer()
    for i in range(0,len(tweets)):
        tweets_sub = []
        for (word,tag) in tweets[i]:
            if tag=='v' or tag =='n':
                tweets_sub.append((lmtzr.lemmatize(word,tag),tag))
            else: 
                tweets_sub.append((word,tag))
        tweets_whole.append(tweets_sub)
    return tweets_whole

lemmatized_tweets = lemmatizer(tagged_tweets_without)


def lemmatizer_untagged(tweets):
    tweets_whole = []
    lmtzr = WordNetLemmatizer()
    for i in range(0,len(tweets)):
        tweets_sub = []
        for (word,tag) in tweets[i]:
            if tag=='v' or tag =='n':
                tweets_sub.append(lmtzr.lemmatize(word,tag))
            else: 
                tweets_sub.append(word)
        tweets_whole.append(tweets_sub)
    return tweets_whole
lemmatized_tweets_untag = lemmatizer_untagged(tagged_tweets_without)
lemmatized_tweets[0]

[(u'over', u'ADP'), (u'dead', u'ADJ'), (u'arrest', 'v')]

### Keeping only NAVA words

In [82]:
def keep_only_nava_words(tagged_tweets):
    """
    :param tagged_tweets:
    :return: tweets_nava
    """
    tweets_nava = []
    tweets_nava_sub = []
    for i in range(0, len(tagged_tweets)):
        for (word, tag) in tagged_tweets[i]:
            if tag == "n" or tag == "v" or tag =="ADJ" or tag == "ADV":
                tweets_nava_sub.append(word)
        tweets_nava.append(list(tweets_nava_sub))
        tweets_nava_sub = []
    return tweets_nava
nava_tweets = keep_only_nava_words(lemmatized_tweets)

### Training word2Vec

In [83]:
def lemmatizer_raw(tweets):
    tweets_whole = []
    lmtzr = WordNetLemmatizer()
    for i in range(0,len(tweets)):
        tweets_sub = []
        for (word,tag) in tweets[i]:
            if tag=='v' or tag =='n':
                tweets_sub.append(lmtzr.lemmatize(word,tag).lower())
            else: 
                tweets_sub.append(word.lower())
        tweets_whole.append(tweets_sub)
    return tweets_whole
raw_tokenized_lemma = lemmatizer_raw(tagged_tweets)
len(raw_tokenized_lemma)
raw_tokenized_lemma[0]

[u'over', u'my', u'dead', u'body', u'are', u'you', u'arresting', u'him']

In [97]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(raw_tokenized_lemma, workers=num_workers, 
            size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "external_tweets_word2vec_model"
model.save(model_name) 

Training model...


In [98]:
model.most_similar('arresting')

KeyError: "word 'arresting' not in vocabulary"